### Starts by defining the problem

In [ ]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce
from math import ceil, exp
from tqdm.auto import tqdm
from tabulate import tabulate
from copy import copy

PROBLEM_SIZE = 1_000 # [100, 1_000, 5_000]
SET_NUMBER = PROBLEM_SIZE
DENSITY = 0.3 # [.3,.7]

def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return np.array(sets.toarray())

In [ ]:
def random_hill_climbing(sets, max_steps=10_000, calling = 0):
    """Returns the number of calls to the fitness function""" 
    calling = 0  
    def fitness(sets,state):
        cost = sum(state)
        valid = np.sum(
            reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
            )
        )
        return valid , -cost , valid == sets.shape[0]

    def tweak(state):
        new_state = copy(state)
        index = randint(0, PROBLEM_SIZE - 1)
        new_state[index] = not new_state[index]
        return new_state

    current_state = [False for _ in range(sets.shape[0])]
    for step in range(max_steps):
        new_state = tweak(current_state)
        fitness_current = fitness(sets,current_state)
        fitness_new = fitness(sets,new_state)
        calling += 2
        if fitness_new >= fitness_current:
            current_state = new_state
        if fitness_current[2]:
            return calling
    return calling

sets = make_set_covering_problem(PROBLEM_SIZE, SET_NUMBER, DENSITY)
max_steps = ceil(SET_NUMBER / 2)
calls = random_hill_climbing(sets , max_steps)
print(f"Set Size = {sets.shape[0]}, fitness calls = {calls}")

In [ ]:
def simulated_annealing(sets, max_steps=10_000, calling = 0,starting_temp = 100):
    """Returns the number of calls to the fitness function""" 
    calling = 0  
    temp = starting_temp

    def tweak_simulated_annealing(current_state : np.array, sets : np.array) -> np.array: 
        """Returns a neighbor of the given state"""
        state = current_state.copy()
        #add a random number of sets 
        #remove a random number of sets
        to_add = randint(1, 5)
        to_remove = randint(1, 5) 
        #print("to_add",to_add,"to_remove", to_remove)     
        added = 0
        removed = 0
        for _ in range(state.shape[0]):
            set_ = randint(0, sets.shape[0]-1)
            if set_ in state and removed < to_remove:
                removed+=1
                state = np.delete(state, np.where(state == set_))
            elif set_ not in state and added < to_add :
                state = np.append(state, set_)
                added += 1
            if added == to_add and removed == to_remove:
                break
        #print("previous :", len(current_state), "current :", len(state))
        return state

    def fitness(sets,state):
        cost = sum(state)
        valid = np.sum(
            reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
            )
        )
        return valid , -cost , valid == sets.shape[0]

    def tweak(state):
        new_state = copy(state)
        index = randint(0, PROBLEM_SIZE - 1)
        new_state[index] = not new_state[index]
        return new_state

    current_state = np.array([False for _ in range(sets.shape[0])])
    fitness_current = None
    for step in range(max_steps):
        if fitness_current is None:
            fitness_current = fitness(sets,current_state)
            calling += 1
        temp = 0.9 * temp
        new_state = tweak_simulated_annealing(current_state,sets)
        fitness_new = fitness(sets,new_state)
        calling += 1
        if exp((fitness_current[0]-fitness_new[0])/temp) >= 0:
            fitness_current = fitness_new
            current_state = new_state
        print(f"Annealing : {exp(-(fitness_current[0]-fitness_new[0])/temp)} , temp : {temp} , fitness_current : {fitness_current[0]} , fitness_new : {fitness_new[0]}")
        if fitness_current[2]:
            return calling , current_state
        
    return calling , current_state

sets = make_set_covering_problem(PROBLEM_SIZE, SET_NUMBER, DENSITY)
max_steps = ceil(SET_NUMBER / 2)
calls , state = simulated_annealing(sets , max_steps)
print(f"Set Size = {sets.shape[0]}, fitness calls = {calls}")
